<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Caso-taller: Identificando  Burger Master con MMG

El Burger Master es un evento creado en el 2016 por el *influencer* Tulio Zuluaga, más conocido en redes como Tulio recomienda, el cual busca que por una semana las hamburgueserías de cada ciudad ofrezcan su mejor producto a un precio reducido.

El evento ha venido creciendo y en el 2022 se extendió por 21 ciudades de Colombia para las cuales se estimó que se vendieron más de dos millones de hamburguesas. El objetivo del presente caso-taller  es identificar los puntos calientes de hamburgueserías  que compitieron en  la ciudad de Bogotá aplicando el Modelo de Mezclas Gaussianas.

## Instrucciones generales

1. Para desarrollar el *cuaderno* primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "Revisión por el compañero."

4. El archivo entregado debe poder ser ejecutado localmente por los pares. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos  en la misma ruta de acceso del cuaderno, por ejemplo: `data`.


## Desarrollo

### 1. Carga de datos  

En la carpeta `data` se encuentra el archivo `burger_master.xlsx` para la ciudad de Bogotá, cargue estos datos en su *cuaderno* y reporte brevemente el contenido de la base.

In [21]:
### librerias
!pip install pyproj folium scikit-learn matplotlib
!pip install folium

import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import pyproj
import folium
from folium.plugins import Search, MarkerCluster
import html
import matplotlib.pyplot as plt
from google.colab import files

In [22]:
df = pd.read_excel("/content/burger_master.xlsx")

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Restaurante  137 non-null    object 
 1   Dirección    137 non-null    object 
 2   Descripción  137 non-null    object 
 3   Latitud      137 non-null    float64
 4   Longitud     137 non-null    float64
dtypes: float64(2), object(3)
memory usage: 5.5+ KB


In [24]:
df.describe(include=['float','object']).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Restaurante,137,63,LA HAMBURGUESERÍA,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dirección,137,116,AV Pradilla # 06 – 95,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Descripción,137,63,<p>#EraGolDeYepes! Burger Master Edition: Una ...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Latitud,137.0,NaN,NaN,NaN,4.677799,0.060859,4.577111,4.643376,4.667144,4.699399,4.892476
Longitud,137.0,NaN,NaN,NaN,-74.074729,0.031671,-74.160764,-74.094575,-74.064046,-74.053926,-74.029229


La base cuenta con 137 registros y 5 columnas las cuales indican el nombre del restaurante, la direccion, descripcion y sus coordenadas.

### 2.  Visualizando los datos

Visualice la ubicación de cada restaurante en un mapa interactivo. Añada un marcador para cada restaurante y la posibilidad de encontrar la descripción de la hamburguesa ofrecida en un pop-up. (Note que la columna Descripción contiene otra información adicional).

In [25]:
# Centro del mapa en la media de coordenadas
center_lat = df['Latitud'].mean()
center_lon = df['Longitud'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Crear los features (para búsqueda y popups)
features = []
for _, row in df.iterrows():
    search_text = f"{row['Restaurante']} {row['Descripción']}"
    features.append({
        "type": "Feature",
        "properties": {
            "Restaurante": row['Restaurante'],
            "Dirección": row['Dirección'],
            "Descripción": row['Descripción'],
            "search": search_text
        },
        "geometry": {
            "type": "Point",
            "coordinates": [row['Longitud'], row['Latitud']]
        }
    })

feature_collection = {"type": "FeatureCollection", "features": features}

# GeoJson layer para búsquedas
geojson_layer = folium.GeoJson(
    feature_collection,
    name="restaurantes-geojson",
    tooltip=folium.GeoJsonTooltip(
        fields=["Restaurante", "Dirección"],
        aliases=["Restaurante:", "Dirección:"]
    )
).add_to(m)

# Marcadores con popups
marker_cluster = MarkerCluster(name="Marcadores").add_to(m)

for feat in features:
    lon, lat = feat["geometry"]["coordinates"]
    props = feat["properties"]
    restaurante = html.escape(str(props["Restaurante"]))
    direccion = html.escape(str(props["Dirección"]))
    descripcion = html.escape(str(props["Descripción"])).replace("\n", "<br>")

    popup_html = f"""
    <b>{restaurante}</b><br>
    <i>{direccion}</i><hr style="margin:6px 0">
    <b>Descripción (detalle):</b><br>{descripcion}
    """

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=350),
        tooltip=restaurante
    ).add_to(marker_cluster)

# Barra de búsqueda
Search(
    layer=geojson_layer,
    search_label="search",
    placeholder="Buscar restaurante o descripción...",
    collapsed=False
).add_to(m)

# Controles de capas
folium.LayerControl().add_to(m)

# # === Exportar a HTML y descargar si quieres abrirlo fuera de Colab ===
# m.save("restaurantes_map.html")
# files.download("restaurantes_map.html")

In [26]:
m

Se construyó un mapa interactivo en Google Colab utilizando la librería Folium, a partir del DataFrame con la información de los restaurantes.
Primero se calculó el punto central de las coordenadas para centrar el mapa y se añadieron los marcadores correspondientes a cada restaurante dentro de un cluster para facilitar la visualización cuando hay puntos cercanos. Cada marcador incluye un popup con el nombre, la dirección y la descripción detallada de la hamburguesa ofrecida. Finalmente, se incorpora una barra de búsqueda que permite localizar rápidamente restaurantes por su nombre o por palabras contenidas en la descripción.

### 3.  Análisis de puntos calientes

Aplique el modelo de Mezclas Gaussianas para buscar clusters de restaurantes en Bogotá, mencione qué estructura de covarianza usó y explique por qué. Escoja el número óptimo de componentes, explicando el procedimiento y justificando su elección.

In [27]:
df_ = df.copy()

# Proyectar coordenadas geográficas a metros (recomendado)
proj_to_m = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
xs, ys = proj_to_m.transform(df_['Longitud'].values, df_['Latitud'].values)
df_['x'] = xs
df_['y'] = ys

# Matriz de features (solo coordenadas en metros)
X = df_[['x','y']].values

# Escalado
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

# Búsqueda de modelo: se prueban varias estructuras de covarianza y número de componentes
cov_types = ['full', 'tied', 'diag', 'spherical']
n_components_range = range(1, 13)

results = []
for cov in cov_types:
    for k in n_components_range:
        gmm = GaussianMixture(n_components=k, covariance_type=cov,
                              n_init=5, random_state=42, reg_covar=1e-6)
        gmm.fit(Xs)
        bic = gmm.bic(Xs)
        aic = gmm.aic(Xs)
        sil = np.nan
        if k > 1:
            labels = gmm.predict(Xs)
            # silhouette en espacio escalado (Euclid)
            try:
                sil = silhouette_score(Xs, labels)
            except:
                sil = np.nan
        results.append({'covariance_type': cov, 'n_components': k,
                        'bic': bic, 'aic': aic, 'silhouette': sil})

res_df = pd.DataFrame(results)
# Mostrar resumen ordenado por BIC
display(res_df.sort_values('bic').head(10))

# Elegir el mejor modelo por BIC (criterio preferido para GMM)
best_idx = res_df['bic'].idxmin()
best_row = res_df.loc[best_idx]
best_cov = best_row['covariance_type']
best_k = int(best_row['n_components'])
print(f"Mejor por BIC -> cov: {best_cov}, componentes: {best_k}, BIC: {best_row['bic']:.1f}, silhouette: {best_row['silhouette']}")

# Ajustar el modelo final con los hiperparámetros elegidos
best_gmm = GaussianMixture(n_components=best_k, covariance_type=best_cov,
                           n_init=10, random_state=42, reg_covar=1e-6)
best_gmm.fit(Xs)
labels = best_gmm.predict(Xs)
df_['cluster'] = labels

# Obtener centroides en coordenadas originales (lon/lat)
means_scaled = best_gmm.means_          # medias en espacio escalado
means_xy = scaler.inverse_transform(means_scaled)  # en metros
# volver a lon/lat
proj_to_lonlat = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
centers_lon, centers_lat = proj_to_lonlat.transform(means_xy[:,0], means_xy[:,1])
centers = list(zip(centers_lat, centers_lon))

# Visualizar en folium (mapa inline en Colab)
center_lat = df_['Latitud'].mean()
center_lon = df_['Longitud'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# color map
import matplotlib.cm as cm
import matplotlib.colors as colors
cmap = cm.get_cmap('tab20', best_k)
norm = colors.Normalize(vmin=0, vmax=best_k-1)

marker_cluster = MarkerCluster().add_to(m)
for i, row in df_.iterrows():
    cl = int(row['cluster'])
    rgba = cmap(cl)
    hexcolor = colors.to_hex(rgba)
    popup_html = f"<b>{html.escape(row['Restaurante'])}</b><br><i>{html.escape(row['Dirección'])}</i><hr><b>Descripción:</b><br>{html.escape(row['Descripción']).replace('\\n','<br>')}"
    folium.CircleMarker(
        location=[row['Latitud'], row['Longitud']],
        radius=5,
        color=hexcolor,
        fill=True,
        fill_opacity=0.8,
        popup=folium.Popup(popup_html, max_width=350),
        tooltip=f"{row['Restaurante']} (cluster {cl})"
    ).add_to(marker_cluster)

# Añadir centroides
for k_i, (latc, lonc) in enumerate(centers):
    folium.CircleMarker(location=[latc, lonc], radius=7,
                        color='black', fill=True, fill_color='white',
                        popup=f"Centro cluster {k_i}").add_to(m)

m  # muestra inline en Colab

# Guardar resultado con cluster en el df original y como CSV
df_with_clusters = df_.copy()
display(df_with_clusters[['Restaurante','Dirección','cluster']].groupby('cluster').count())

,covariance_type,n_components,bic,aic,silhouette
10,full,11,564.924905,375.126145,0.471738
46,spherical,11,574.599196,449.040016,0.481395
11,full,12,574.705824,367.387179,0.448118
47,spherical,12,580.073974,442.834871,0.493332
34,diag,11,582.729948,425.050979,0.481507
35,diag,12,589.034988,416.756114,0.499687
9,full,10,600.862637,428.583762,0.418918
41,spherical,6,604.581399,537.421838,0.457890
4,full,5,608.625123,523.945677,0.398700
42,spherical,7,608.697599,529.858114,0.392507


Mejor por BIC -> cov: full, componentes: 11, BIC: 564.9, silhouette: 0.4717384188164917


/tmp/ipython-input-108674419.py:73: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', best_k)


,Restaurante,Dirección
cluster,,
0,48,48
1,11,11
2,2,2
3,16,16
4,16,16
5,8,8
6,4,4
7,8,8
8,5,5


Se aplicó un Modelo de Mezclas Gaussianas (GMM) sobre las coordenadas de los restaurantes en Bogotá, previamente transformadas a un sistema de proyección en metros para garantizar que las distancias tuvieran una escala realista.
Se evaluaron cuatro estructuras de covarianza (full, tied, diag, spherical) y un rango de 1 a 12 componentes. El criterio principal de selección fue el BIC, complementado con el índice de silhouette.
El mejor modelo resultó con 11 componentes y covarianza full, alcanzando un BIC de aproximadamente 564.9 y una puntuación de silhouette de 0.47. La elección de la estructura full se justifica porque permite que cada cluster tenga su propia matriz de covarianza, capturando formas elípticas de distinta orientación y dispersión, lo que refleja mejor la heterogeneidad espacial de la ciudad. Los clusters obtenidos representan concentraciones geográficas de restaurantes en distintas zonas de Bogotá, aportando una segmentación útil para posteriores análisis de localización y oferta gastronómica.

#### 3.1. Visualización de los resultados

Visualice las densidades estimadas por el  mejor modelo estimado en la sección anterior usando un mapa de calor interactivo, discuta los resultados.

In [28]:
import numpy as np
from folium.plugins import HeatMap

# Definir la malla de puntos en el área de Bogotá
lat_min, lat_max = df['Latitud'].min(), df['Latitud'].max()
lon_min, lon_max = df['Longitud'].min(), df['Longitud'].max()

# Generar grilla regular
grid_size = 100  # mayor = más detallado, pero más lento
lats = np.linspace(lat_min, lat_max, grid_size)
lons = np.linspace(lon_min, lon_max, grid_size)
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Convertir a coordenadas proyectadas (metros), igual que antes
proj_to_m = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
xs, ys = proj_to_m.transform(lon_grid.ravel(), lat_grid.ravel())
XY_grid = np.vstack([xs, ys]).T

# Escalar con el mismo scaler usado para entrenar el GMM
XYs = scaler.transform(XY_grid)

# Evaluar la densidad logarítmica y convertir a probabilidad
log_dens = best_gmm.score_samples(XYs)
dens = np.exp(log_dens)

# Normalizar para que los pesos estén entre 0 y 1
dens_norm = (dens - dens.min()) / (dens.max() - dens.min())

# Construir puntos con intensidad
heat_data = [
    [lat_grid.ravel()[i], lon_grid.ravel()[i], float(dens_norm[i])]
    for i in range(len(dens_norm))
]

# Crear mapa con capa de calor
m_heat = folium.Map(location=[df['Latitud'].mean(), df['Longitud'].mean()], zoom_start=12)
HeatMap(heat_data, radius=15, blur=25, max_zoom=13).add_to(m_heat)

# Mostrar en Colab
m_heat

- El mapa de calor refleja la densidad de probabilidad estimada por el GMM, es decir, las áreas de Bogotá donde es más probable encontrar restaurantes según la distribución aprendida.

- Los picos de calor corresponden a los centroides de los 11 clusters detectados. En la práctica, esto marca las zonas con mayor concentración de restaurantes, como corredores gastronómicos o centros comerciales.

- La elección de covarianza full permite que cada zona tenga una forma elíptica distinta, por lo que algunos focos del mapa de calor aparecen alargados o dispersos, mientras otros son más compactos.

### 4. Comparación con KDE

Estime ahora las densidades usando KDE bivariado de la librería `statsmodels` con el anchos de banda dado por `cv_ml`. Muestre los resultados usando un mapa interactivo. Compare los resultados obtenidos por el "mejor" modelo encontrado via MMG.

In [29]:
import statsmodels.api as sm
from folium.plugins import HeatMap

# Tomar coordenadas de restaurantes y transformarlas (proyección métrica)
X = df[['Longitud', 'Latitud']].to_numpy()
xs, ys = proj_to_m.transform(X[:,0], X[:,1])
data = np.vstack([xs, ys])

# KDE bivariado con selección de ancho de banda cv_ml
kde = sm.nonparametric.KDEMultivariate(data=data.T, var_type='cc', bw='cv_ml')

# Generar grilla sobre Bogotá
lat_min, lat_max = df['Latitud'].min(), df['Latitud'].max()
lon_min, lon_max = df['Longitud'].min(), df['Longitud'].max()

grid_size = 200
lats = np.linspace(lat_min, lat_max, grid_size)
lons = np.linspace(lon_min, lon_max, grid_size)
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Pasar grilla a proyección métrica
xs_grid, ys_grid = proj_to_m.transform(lon_grid.ravel(), lat_grid.ravel())
grid_points = np.vstack([xs_grid, ys_grid]).T

# Evaluar densidad KDE
dens_kde = kde.pdf(grid_points)

# Normalizar (0 a 1 para HeatMap)
dens_norm = (dens_kde - dens_kde.min()) / (dens_kde.max() - dens_kde.min())

heat_data_kde = [
    [lat_grid.ravel()[i], lon_grid.ravel()[i], float(dens_norm[i])]
    for i in range(len(dens_norm))
]

# Crear mapa interactivo
m_kde = folium.Map(location=[df['Latitud'].mean(), df['Longitud'].mean()], zoom_start=12)
HeatMap(heat_data_kde, radius=15, blur=25, max_zoom=13).add_to(m_kde)

m_kde

/usr/local/lib/python3.12/dist-packages/statsmodels/nonparametric/kernel_density.py:158: RuntimeWarning: invalid value encountered in log
  L += func(f_i)


Comparación con el modelo de Mezclas Gaussianas (GMM)

- El KDE no asume un número fijo de clusters ni formas predefinidas; ajusta la densidad de manera suave y continua a partir de los datos. GMM en cambio busca aproximar la distribución como una suma de gaussianas; en este caso, con 11 componentes.


- GMM permite identificar clusters concretos (útil para segmentación de zonas), mientras que KDE es mejor para obtener una estimación global de densidad (útil para mapas de calor continuos).

**Conclusiones:**

El KDE da una visión más natural y menos forzada de cómo se distribuyen los restaurantes en Bogotá.

El GMM en cambio es más potente si la meta es identificar agrupamientos discretos y cuantificables.

Usados en conjunto, aportan dos perspectivas: zonas densas (KDE) y clusters estructurados (GMM).